In [1]:
import pandas as pd 

In [3]:
data = pd.read_csv("Data/Train_Data_updated.csv")

In [4]:
from Utils import preprocess, analyze,misc

In [5]:
data.head()

,Unnamed: 0,Subtitle,Emotion,Analyzer_Output,New_Emotion
0,0,i didnt feel humiliated,sadness,Neutral,Sadness
1,1,i can go from feeling so hopeless to so damned...,sadness,Negative,Despair
2,2,im grabbing a minute to post i feel greedy wrong,anger,Negative,Resentment
3,3,i am ever feeling nostalgic about the fireplac...,love,Neutral,Love
4,4,i am feeling grouchy,anger,Negative,Resentment


In [6]:
from sklearn.model_selection import train_test_split
X = data['Subtitle']
y = data['New_Emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [9]:
tokenizer = preprocess.tokenizer
tokenizer_port = preprocess.tokenizer_port

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{"vect__ngram_range": [(1, 1)],
              "vect__stop_words": [stop, None],
               "clf__penalty": ['l1', 'l2'],
               "clf__C": [1.0, 10.0, 100.0],
               "vect__tokenizer": [tokenizer, tokenizer_port]},
              {"vect__ngram_range": [(1, 1)],
              "vect__stop_words": [stop, None],
               "vect__tokenizer": [tokenizer, tokenizer_port],
               "vect__norm": [None],
               "vect__use_idf": [False],
               "clf__penalty": ['l1', 'l2'],
               "clf__C": [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([("vect", tfidf),
                    ("clf", LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [11]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Users\anisr\anaconda3\envs\work\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x00000170268A3280>,
                                              <function tokenizer_port at 0x00000170375A9C10>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=1)

In [12]:
model = gs_lr_tfidf.best_estimator_
print("Train accuracy: ", model.score(X_train, y_train))
print("Test accuracy: ", model.score(X_test,y_test))

Train accuracy:  0.8214369158878505
Test accuracy:  0.697196261682243


In [14]:
model.predict(["what am i supposed to do now?"])

array(['Calm'], dtype=object)